In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from tqdm import tqdm
import sys
import matplotlib as mpl

#insert path
sys.path.insert(0, '../methods/')

sys.modules.pop('generate_syn_data', None)
from generate_syn_data import *

sys.modules.pop('ARWQE', None)
from ARWQE import *

sys.modules.pop('plots', None)
from plots import *

In [4]:
task = 'linreg' #linear regression task

num_trials = 1 #NOTE: change this to run multiple trials

num_periods = 1000
alpha =0.1; delta = 0.1; gamma=1
dimX = 5; variance_y = 1; meanX = 0; 

fixed_windows = [1, 4, 16, 64, 256, 1024]
train_windows = [1, 64, 256, 1024]
rho_values = [0.99, 0.9, 0.5, 0.25]


#NOTE: choose 'stationary' or 'nonstationary'
shift = 'nonstationary' 

if shift == 'stationary':
    beta = np.ones((num_periods, dimX)) /5
elif shift == 'nonstationary':
    #non-stationary case
    beta_1 = generate_true_means(num_periods-1, 40, 10)
    beta_2 = generate_true_means(num_periods-1, 40, 10)
    beta_3 = generate_true_means(num_periods-1, 40, 10)
    beta_4 = generate_true_means(num_periods-1, 40, 10)
    beta_5 = generate_true_means(num_periods-1, 40, 10)
    beta = 2 * np.column_stack((beta_1, beta_2, beta_3, beta_4, beta_5)) 

np.random.seed(6)

#B_arr is for val set
B_arr = np.random.randint(low=1, high=10, size=num_periods)
B_arr_starts = np.arange(num_periods)
B_arr_ends = np.cumsum(B_arr) - 1

#for training set
B_arr_tr = 3 * B_arr

cdf_array = np.empty((num_trials, len(fixed_windows)+1+len(rho_values), len(train_windows), num_periods))
interval_array = np.empty((num_trials, len(fixed_windows)+1+len(rho_values), len(train_windows), num_periods))

seeds = np.arange(num_trials) + 2024

for (trial, seed) in tqdm(enumerate(seeds)):

    np.random.seed(seed)

    X_tr, y_tr = generate_linreg_data(meanX, B_arr_tr, beta, variance_y)
    X_val, y_val = generate_linreg_data(meanX, B_arr, beta, variance_y)

    for t in range(num_periods):

        tr_end = B_arr_ends[t]+1
        B_arr_t = B_arr[:t+1]

        for m, train_window in enumerate(train_windows):

            # use fixed windows for training
            tr_start = B_arr_starts[t-min(t,train_window)]
            reg, S_t = fit_and_get_scores(X_tr[tr_start:tr_end], y_tr[tr_start:tr_end],\
                                           X_val[:tr_end], y_val[:tr_end])

            khat, qt_khat, qtk_all = ARWQE(S_t, B_arr_t, alpha, delta, gamma)

            #compute coverage
            dimX = X_tr.shape[1]

            #approximate by generating ~1000 X, Y from this period
            # and calculate coverage of prediction set
            X_test = generate_multinomial_X(meanX, dimX, 1000)
            y_hat = reg.predict(X_test)

            mu_test = X_test @ beta[t].T
            Y_test = mu_test + np.random.normal(0, variance_y, 1000)

            coverage_ARW = monte_carlo_coverage(y_hat, qt_khat, Y_test)
            cdf_array[trial, 0, m, t] = coverage_ARW
            interval_array[trial, 0, m, t] = 2*qt_khat

            #baseline: weighted quantile
            for r, rho in enumerate(rho_values):
              qw = QE_weighted(S_t, B_arr_t, alpha, rho)
              coverage_w = monte_carlo_coverage(y_hat, qw, Y_test)
              cdf_array[trial, 1+r, m, t] = coverage_w
              interval_array[trial, 1+r, m, t] = 2*qw

            #baseline: take quantile of fixed k
            for ik, k in enumerate(fixed_windows):
                log2k = int(np.log2(k))
                qtk = qtk_all[min(log2k, len(qtk_all)-1)]
                coverage_k = monte_carlo_coverage(y_hat, qtk, Y_test)
                cdf_array[trial, ik+1+len(rho_values), m, t] = coverage_k
                interval_array[trial, ik+1+len(rho_values), m, t] = 2*qtk



1it [00:17, 17.96s/it]


In [5]:
np.save(f'./results/{task}_{shift}_cdf_array_rhos.npy', cdf_array, allow_pickle=True)